In [53]:
# import models
import mplfinance as mpf
from config import api_key
import requests
import pandas as pd
import time
import datetime
from datetime import time
import matplotlib.pyplot as plt
import cufflinks as cf
import os


In [75]:
#input info used to pass through api 
symbol = input("What ticker do you want to search?")
periodtype = input("Period type (day, month, year)")
frequency = input("Frequency of data")
frequencytype = input("Frequency type (minute, monthly, daily)")
period = input("Period of data")
needExtendedHoursData = input('Extended hours data (True or False)')


What ticker do you want to search?OXY
Period type (day, month, year)year
Frequency of data1
Frequency type (minute, monthly, daily)monthly
Period of data20
Extended hours data (True or False)False


In [76]:
#url for td
url = "https://api.tdameritrade.com/v1/marketdata/{}/pricehistory".format(symbol)
    
#parameters
key = {'apikey':api_key,
         'periodType':periodtype,
          'frequencyType': frequencytype,
          'frequency': frequency,
          'period': period,
          'needExtendedHoursData': needExtendedHoursData,
         }
#requesting data
content = requests.get(url = url, params=key)

data = content.json()
data

{'candles': [{'open': 14.975,
   'high': 15.04,
   'low': 11.49,
   'close': 13.545,
   'volume': 83961800,
   'datetime': 1025499600000},
  {'open': 13.525,
   'high': 15.0,
   'low': 12.465,
   'close': 14.85,
   'volume': 51088800,
   'datetime': 1028178000000},
  {'open': 14.65,
   'high': 14.95,
   'low': 13.715,
   'close': 14.19,
   'volume': 63683400,
   'datetime': 1030856400000},
  {'open': 14.45,
   'high': 15.37,
   'low': 13.76,
   'close': 14.265,
   'volume': 86534600,
   'datetime': 1033448400000},
  {'open': 14.175,
   'high': 14.43,
   'low': 13.235,
   'close': 13.925,
   'volume': 53321200,
   'datetime': 1036130400000},
  {'open': 13.95,
   'high': 14.62,
   'low': 13.565,
   'close': 14.225,
   'volume': 51116200,
   'datetime': 1038722400000},
  {'open': 14.15,
   'high': 14.785,
   'low': 13.585,
   'close': 14.605,
   'volume': 52401800,
   'datetime': 1041400800000},
  {'open': 14.605,
   'high': 14.965,
   'low': 14.22,
   'close': 14.95,
   'volume': 7560560

In [77]:
#create arrays for each category
open_tick = []
high_tick = []
low_tick = []
close_tick = []
volume_tick = []
datetime_tick = []
#iterrates through data to append each array with it's value
for i in range(len(data["candles"])):
    open_tick.append(data["candles"][i]["open"])
    high_tick.append(data["candles"][i]["high"])
    low_tick.append(data["candles"][i]["low"])
    close_tick.append(data["candles"][i]["close"])
    volume_tick.append(data["candles"][i]["volume"])
    datetime_tick.append(data["candles"][i]["datetime"])

#converting epoch to datetime       
file = []
for i in range(len(datetime_tick)):
    # divide epoch by 1000 because it is in milliseconds
    test = datetime_tick[i]/1000
    file.append(datetime.datetime.fromtimestamp(test))

#convert to dataframe
theta_df = pd.DataFrame({
            "Open":open_tick,
            "Close":close_tick,
            "High":high_tick,
            "Low":low_tick,
            "Volume":volume_tick,
            "Datetime": file
    })

In [78]:
# set index to Datetime
theta_df = theta_df.set_index("Datetime")

In [79]:
# Calculate % gain for the day 
theta_df["Gain%"] = (theta_df.Close - theta_df.Open)/theta_df.Open * 100

theta_df

,Open,Close,High,Low,Volume,Gain%
Datetime,,,,,,
2002-06-30 22:00:00,14.975,13.545,15.04,11.4900,83961800,-9.549249
2002-07-31 22:00:00,13.525,14.850,15.00,12.4650,51088800,9.796673
2002-08-31 22:00:00,14.650,14.190,14.95,13.7150,63683400,-3.139932
2002-09-30 22:00:00,14.450,14.265,15.37,13.7600,86534600,-1.280277
2002-10-31 22:00:00,14.175,13.925,14.43,13.2350,53321200,-1.763668
...,...,...,...,...,...,...
2022-01-31 22:00:00,37.250,43.730,43.78,37.1800,417232093,17.395973
2022-02-28 22:00:00,44.330,56.740,63.24,44.0400,1245250338,27.994586
2022-03-31 22:00:00,56.820,55.090,63.56,51.5300,498547135,-3.044703


In [80]:
# for the stock ticker if a folder does not exist already, one will be created
mypath = f"{symbol}"
if not os.path.isdir(mypath):
   os.makedirs(mypath)

In [81]:
# uploads csv to that folder specifying the frequency and type to identify the csv
upload_file = f"{symbol}/{symbol}_{frequencytype}_{frequency}_{needExtendedHoursData}.csv"
theta_df.to_csv(upload_file, index = True)